In [35]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

In [76]:
df1 = pd.read_csv('input/2012/qb/current_qb_stats.csv')
df2 = pd.read_csv('input/2012/qb/recent_qb_2012.csv')
df3 = pd.read_csv('input/2012/qb/career_qb_stats.csv')

In [77]:
df2.drop('fantasy points_ma', axis =1, inplace=True)

In [38]:
week = df1.Week
y=df1.points
opp = df1.Opp
df1.drop(['Week','points','Opp'], axis=1, inplace =True)

df2.drop(['Week','Opp'], axis=1, inplace=True)
df3.drop(['Week','Opp'], axis=1, inplace=True)

In [182]:
y=df1.points


In [75]:
df1.shape,df2.shape,df3.shape

((629, 17), (629, 13), (629, 13))

In [40]:
df3.head() 

,Player,comp_car,att_car,pct_car,yds_car,yds/att_car,td_car,int_car,qb rating_car,rush_att_car,rush_yds_car,rush_yd/att_car,rush_td_car
0,Aaron Rodgers,30.00,44.000000,68.0,303.000000,7.000000,2.00,1.000000,93.000000,5.000000,27.000000,5.00,0.0
1,Aaron Rodgers,26.00,38.000000,68.5,261.000000,7.000000,1.50,1.000000,89.000000,4.000000,10.500000,1.50,0.0
2,Aaron Rodgers,26.00,38.333333,68.0,248.333333,6.666667,1.00,0.666667,86.333333,3.333333,12.666667,4.00,0.0
3,Aaron Rodgers,27.25,39.000000,70.0,266.000000,7.000000,1.75,0.750000,94.750000,3.750000,12.750000,3.75,0.0
4,Aaron Rodgers,25.80,37.600000,68.6,259.800000,7.000000,2.00,0.800000,96.400000,4.000000,21.600000,5.20,0.0


In [41]:
df2.head()

,Player,comp_ma,att_ma,pct_ma,yds_ma,yds/att_ma,td_ma,int_ma,qb rating_ma,rush_att_ma,rush_yds_ma,rush_yd/att_ma,rush_td_ma
0,Aaron Rodgers,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0
1,Aaron Rodgers,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0
2,Aaron Rodgers,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0
3,Aaron Rodgers,26.00,38.333333,68.0,248.333333,6.666667,1.00,0.666667,86.333333,3.333333,12.666667,4.00,0.0
4,Aaron Rodgers,27.25,39.000000,70.0,266.000000,7.000000,1.75,0.750000,94.750000,3.750000,12.750000,3.75,0.0


In [71]:
df = pd.concat([df1,df2],axis=1)

In [72]:
df = pd.concat([df,df3], axis=1)

In [78]:
def nfl_position_lr(df1,df2,df3):
    df = pd.merge(df1, df2, how='outer')
    df=pd.merge(df, df3, how='outer')
    df.reset_index(inplace=True, drop=True)
    return df


In [163]:
qb = nfl_position_lr(df1,df2,df3)

In [73]:
df.shape

(629, 43)

In [51]:
qb = df

In [164]:
qb.columns

Index(['Player', 'Week', 'Opp', 'avg_comp', 'avg_att', 'avg_pct', 'avg_yds',
       'avg_yds/att', 'avg_td', 'avg_int', 'avg_qb rating', 'avg_rush_att',
       'avg_rush_yds', 'avg_rush_yd/att', 'avg_rush_td', 'season', 'def_ru_rk',
       'def_pass_rk', 'week', 'points', 'comp_ma', 'att_ma', 'pct_ma',
       'yds_ma', 'yds/att_ma', 'td_ma', 'int_ma', 'qb rating_ma',
       'rush_att_ma', 'rush_yds_ma', 'rush_yd/att_ma', 'rush_td_ma',
       'comp_car', 'att_car', 'pct_car', 'yds_car', 'yds/att_car', 'td_car',
       'int_car', 'qb rating_car', 'rush_att_car', 'rush_yds_car',
       'rush_yd/att_car', 'rush_td_car'],
      dtype='object')

In [74]:
qb['week'] = week


In [ ]:
qb.groupby

In [169]:
qb.index = qb['Player']
qb.drop('Player', axis=1,inplace=True)

In [84]:
qb.shape

(629, 43)

In [58]:
#A function that will covert a time series database into a supervised learning database

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
            cols.append(df.shift(i))
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [167]:
qb.head()

,Player,avg_comp,avg_att,avg_pct,avg_yds,avg_yds/att,avg_td,avg_int,avg_qb rating,avg_rush_att,...,pct_car,yds_car,yds/att_car,td_car,int_car,qb rating_car,rush_att_car,rush_yds_car,rush_yd/att_car,rush_td_car
0,Aaron Rodgers,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,68.0,303.000000,7.000000,2.00,1.000000,93.000000,5.000000,27.000000,5.00,0.0
1,Aaron Rodgers,26.00,38.000000,68.5,261.000000,7.000000,1.50,1.000000,89.000000,4.000000,...,68.5,261.000000,7.000000,1.50,1.000000,89.000000,4.000000,10.500000,1.50,0.0
2,Aaron Rodgers,26.00,38.333333,68.0,248.333333,6.666667,1.00,0.666667,86.333333,3.333333,...,68.0,248.333333,6.666667,1.00,0.666667,86.333333,3.333333,12.666667,4.00,0.0
3,Aaron Rodgers,27.25,39.000000,70.0,266.000000,7.000000,1.75,0.750000,94.750000,3.750000,...,70.0,266.000000,7.000000,1.75,0.750000,94.750000,3.750000,12.750000,3.75,0.0
4,Aaron Rodgers,25.80,37.600000,68.6,259.800000,7.000000,2.00,0.800000,96.400000,4.000000,...,68.6,259.800000,7.000000,2.00,0.800000,96.400000,4.000000,21.600000,5.20,0.0


In [166]:
y = qb.points.values
opp = qb.Opp.values
week = qb['Week'].values
drush = qb.def_ru_rk.values
dpass = qb.def_pass_rk.values
qb.drop(['Opp','Week','def_ru_rk','def_pass_rk','points'], axis =1, inplace=True)

In [170]:
df = qb.astype('float32')

In [171]:
# scale data, reframe for supervised learning and drop target variable
scaler = MinMaxScaler(feature_range=(0,1))
scaled  = scaler.fit_transform(df)
reframed = series_to_supervised(scaled, 1, 1)
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),...,var29(t),var30(t),var31(t),var32(t),var33(t),var34(t),var35(t),var36(t),var37(t),var38(t)
1,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.056452,...,0.685,0.721992,0.350000,0.500000,0.250000,0.563291,0.363636,0.204637,0.184211,0.0
2,0.912281,0.752475,0.685,0.721992,0.350000,0.500000,0.333333,0.563291,0.363636,0.204637,...,0.680,0.686952,0.333333,0.333333,0.166667,0.546414,0.303030,0.235215,0.315789,0.0
3,0.912281,0.759076,0.680,0.686952,0.333333,0.333333,0.222222,0.546414,0.303030,0.235215,...,0.700,0.735823,0.350000,0.583333,0.187500,0.599684,0.340909,0.236391,0.302632,0.0
4,0.956140,0.772277,0.700,0.735823,0.350000,0.583333,0.250000,0.599684,0.340909,0.236391,...,0.686,0.718672,0.350000,0.666667,0.200000,0.610127,0.363636,0.361290,0.378947,0.0
5,0.905263,0.744554,0.686,0.718672,0.350000,0.666667,0.266667,0.610127,0.363636,0.361290,...,0.680,0.754726,0.366667,0.888889,0.166667,0.649789,0.333333,0.350470,0.412281,0.0


In [179]:
reframed['dp_rk'] = dpass[1:]
reframed['drush'] = drush[1:]

In [184]:
X = reframed.values
y= y[1:]

In [185]:
y.shape

(628,)

In [176]:
len(X)

628

In [186]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [187]:
yhat=lr.predict(X)
yhat[0:30]

array([  4.90564355,   8.70102883,  25.04531481,  21.78248599,
        29.12025728,  20.49196042,  17.41771909,  19.73209687,
        18.51572812,  17.06000945,  16.83966697,  18.18284463,
        17.80553915,  22.66389115,  21.66429228,   9.51423641,
        16.63634314,  14.42059515,   8.62859269,  24.17801675,
         8.91236757,   9.37615301,  16.94361942,  11.61289588,
        10.82848929,  12.66005901,  10.30164794,   9.91618858,
         9.57999256,   7.61285091])

In [188]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

rmse_val = rmse(yhat, y)
print("RMSE error is: " + str(rmse_val))

RMSE error is: 4.61518440573


In [189]:
rmse_val/np.mean(yhat)

0.36604115235259188

In [159]:
week.values

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 14, 15, 16, 17,  1,
        2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17,  1,  2,
        3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,  1,  2,  3,
        4,  5,  6,  7,  9, 10, 11, 12, 13, 14, 15, 16, 17,  1,  2,  3,  5,
        6,  7,  8,  9, 10, 14, 15, 16, 17,  1,  2,  3,  4,  5,  7,  8,  9,
       10, 11,  5,  6,  8, 11, 12, 13, 14, 15, 16, 17,  1,  2,  3,  4,  5,
        6,  7,  8,  9, 11, 12, 13, 14, 15, 16, 16, 17,  4,  8, 15, 16, 17,
        1, 17, 10, 11,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14,
       15, 16, 17,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16,  2,  7, 10, 11, 12, 13, 14, 15, 16, 17, 12, 13,  1,  2,  3,  4,
        5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,  1,  2,  3,  4,  5,
        6,  7,  8,  9, 10, 12, 13, 14, 15, 16, 17,  1,  4,  5,  6,  7, 10,
       11, 12, 13, 14, 15, 16, 17, 12, 14, 16, 15,  3, 17,  3, 17,  1,  2,
        3,  4,  5,  7,  8

In [111]:
qb.index.shape

(629,)

In [118]:
yhat.shape

(628,)

In [139]:
y.drop(['level_0','index'], axis=1, inplace=True)

In [143]:
y.drop('Player', axis=1,inplace=True)

In [196]:
df1.columns

Index(['Player', 'Week', 'Opp', 'avg_comp', 'avg_att', 'avg_pct', 'avg_yds',
       'avg_yds/att', 'avg_td', 'avg_int', 'avg_qb rating', 'avg_rush_att',
       'avg_rush_yds', 'avg_rush_yd/att', 'avg_rush_td', 'season', 'def_ru_rk',
       'def_pass_rk', 'week', 'points'],
      dtype='object')

In [219]:
results = pd.DataFrame()
results['Player'] = qb.index[1:]
results['score'] = y[1:]
results['Predicted'] = yhat
results.groupby('Player').mean()[['score','Predicted']]
#results['week'] = df1['Week']

,score,Predicted
Player,,
Aaron Rodgers,21.770769,18.661899
Alex Smith,9.452500,11.795485
Andrew Luck,16.297500,17.500249
Andy Dalton,16.186250,17.341139
Ben Roethlisberger,16.473846,17.766272
Blaine Gabbert,11.228000,10.739678
Brady Quinn,4.084000,4.523268
Brandon Weeden,10.893333,10.493957
Brian Hoyer,3.380000,4.050586


In [211]:
results.shape

(628, 4)

In [215]:
results['week'] = df1['Week'][2:]

In [218]:
results

,Player,score,Predicted,week
0,Aaron Rodgers,NaN,4.905644,1
1,Aaron Rodgers,NaN,8.701029,2
2,Aaron Rodgers,10.62,25.045315,3
3,Aaron Rodgers,28.06,21.782486,4
4,Aaron Rodgers,25.10,29.120257,5
5,Aaron Rodgers,39.22,20.491960,6
6,Aaron Rodgers,25.68,17.417719,7
7,Aaron Rodgers,13.84,19.732097,8
8,Aaron Rodgers,26.02,18.515728,9
9,Aaron Rodgers,15.74,17.060009,11
